In [39]:
import Config
import Stats
import Game
import ScoreBoard
import BoxScore

import importlib
importlib.reload(Config)
importlib.reload(Stats)
importlib.reload(Game)
importlib.reload(ScoreBoard)
importlib.reload(BoxScore)

from ipywidgets import widgets, HTML
output = widgets.Output()
display(output)

import pandas as pd
import glob
import os
import datetime

allfiles = glob.glob('./data/*.game')
phases = sorted(list(set([os.path.basename(x).split('-')[1] for x in allfiles if '-' in x and '.a-' in x])))

allevents = []
sb = ScoreBoard.ScoreBoard('./data/Urbania.team', scale=0.4, output=output)

# Conversion to int without errors
def toint(x):
    try:
        return int(x)
    except:
        return 999999

players_info = {}

html_head = '''<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <meta name="Robots" content="index,follow" />
    <meta name="author" content="Davide De Marchi" />
    <meta name="description" content="pyBasket" />
    <meta name="keywords" content="key, words" />
    <link rel="stylesheet" type="text/css" href="css/layout.css" media="screen" />
    <link rel="icon" href="basket.ico" />
    <title>Statistiche %s - %s - pyBasket</title>
    <style>
        body {font-family: Arial;}

        /* Style the horizontal tab */
        .tab {
          overflow: hidden;
          width: 84%%;
        }
        
        /* Style the buttons inside the tab */
        .tab button {
          background-color: inherit;
          float: left;
          border: none;
          outline: none;
          cursor: pointer;
          margin-top: 16px; 
          padding: 22px 16px;
          transition: 0.3s;
          font-size: 24px;
        }
        
        /* Change background color of buttons on hover */
        .tab button:hover {
          background-color: #ddd;
        }
        
        /* Create an active/current tablink class */
        .tab button.active {
          color: white;
          background-color: #b82020;
        }
        
        /* Style the tab content */
        .tabcontent {
          display: none;
          padding: 6px 12px;
        }

        #left {
            width: 300px;
            float: left;
        }
        #right {
            width: 600px;
            float: left;
        }
        td:hover {
            cursor:pointer;
        }
        
        /* Style the vertical tab */
        .vtab {
            float: left;
            width: 100px;
        }

        /* Style the buttons inside the tab */
        .vtab button {
            display: block;
            background-color: inherit;
            color: black;
            padding: 22px 16px;
            width: 100%%;
            border: none;
            outline: none;
            text-align: left;
            cursor: pointer;
            transition: 0.3s;
            font-size: 17px;
        }

        /* Change background color of buttons on hover */
        .vtab button:hover {
            background-color: #ddd;
        }

        /* Create an active/current "tab button" class */
        .vtab button.active {
            color: white;
            background-color: #b82020;
        }

        /* Style the tab content */
        .vtabcontent {
            float: left;
            padding: 0px 12px;
            width: calc(100%% - 130px);
            border-left: none;
        }
        
        %s
        </style>
        
        <script>
        function includeHTML() {
          var z, i, elmnt, file, xhttp;
          /*loop through a collection of all HTML elements:*/
          z = document.getElementsByTagName("*");
          for (i = 0; i < z.length; i++) {
            elmnt = z[i];
            /*search for elements with a certain attribute:*/
            file = elmnt.getAttribute("w3-include-html");
            if (file) {
              /*make an HTTP request using the attribute value as the file name:*/
              xhttp = new XMLHttpRequest();
              xhttp.onreadystatechange = function() {
                if (this.readyState == 4) {
                  if (this.status == 200) {elmnt.innerHTML = this.responseText;}
                  if (this.status == 404) {elmnt.innerHTML = "Page not found.";}
                  /*remove the attribute, and call this function once more:*/
                  elmnt.removeAttribute("w3-include-html");
                  includeHTML();
                }
              }      
              xhttp.open("GET", file, true);
              xhttp.send();
              /*exit the function:*/
              return;
            }
          }
        };
        </script>
        
</head>
<body>
    <div id="content">
        <p id="top">Tabellini statistici e mappe di tiro di tutti i giocatori</p>
        <div id="logo">
            <h1><a href="index.html">%s</a></h1>
        </div>
        <div class="line"></div>
        <div id="pitch">
            <div id="left">
                <h5>
                Data di affiliazione: 1/7/1977<br />
                Sede: Piazza Nicolò Pellipario - Urbania (PU)<br />
                Telefono: +39 339 172 1198<br />
                Mail: <a href="mailto:uspurbania@libero.it">uspurbania@libero.it</a><br />
                Colori sociali: Bianco e Rosso<br />
                </h5>
            </div>
            
            <div id="right">
                <h2>%s - Stagione %s</h2>
            </div>
            <div class="tab">
                <button class="tablinks" onclick="openContent(event, 'Partite')" id="defaultOpen">Partite</button>
                <button class="tablinks" onclick="openContent(event, 'Totali')">Totali</button>
                <button class="tablinks" onclick="openContent(event, 'Medie')">Medie</button>
            </div>
        </div>

        <div id="Totali" class="tabcontent">
            <embed src="totali.svg"  width="100%%" height="900px"/>
        </div>

        <div id="Medie" class="tabcontent">
            <embed src="medie.svg"  width="100%%" height="900px"/>
        </div>
'''



html_vtabcontent = '''
        .vtabcontent%d {
            float: left;
            padding: 0px 12px;
            width: calc(100%% - 130px);
            border-left: none;
        }
'''

html_function = '''
        function openGame%d(evt, name) {
            var i, vtabcontent, vtablinks;
            vtabcontent = document.getElementsByClassName("vtabcontent%d");
            for (i = 0; i < vtabcontent.length; i++) {
                vtabcontent[i].style.display = "none";
            }
            vtablinks = document.getElementsByClassName("vtablinks%d");
            for (i = 0; i < vtablinks.length; i++) {
                vtablinks[i].className = vtablinks[i].className.replace(" active", "");
            }
            document.getElementById(name).style.display = "block";
            evt.currentTarget.className += " active";
        }
        
        // Get the element with id="vdefaultOpen" and click on it
        document.getElementById("vdefaultOpen%d").click();
'''

html_singlegame = '''
        <div id="p%d" class="tabcontent">
            <div class="vtab">
              <button class="vtablinks%d" onclick="openGame%d(event, 'Tabellino%d')" id="vdefaultOpen%d">Tabellino</button>
              <button class="vtablinks%d" onclick="openGame%d(event, 'Cronaca%d')">Cronaca</button>
              <button class="vtablinks%d" onclick="openGame%d(event, 'Mappe%d')">Mappe</button>
              <button class="vtablinks%d" onclick="openGame%d(event, 'Grafico%d')">Grafico</button>
              <button class="vtablinks%d" onclick="openGame%d(event, 'Sintesi%d')">Sintesi</button>
            </div>

            <div id="Tabellino%d" class="vtabcontent%d">
              <embed src="%d.svg"  width="100%%" height="800px"/>
            </div>
              
            <div id="Cronaca%d" class="vtabcontent%d">
                <div w3-include-html="%d.html"></div> 
            </div>
              
            <div id="Mappe%d" class="vtabcontent%d">
            </div>
            
            <div id="Grafico%d" class="vtabcontent%d">
                <img src="%d.png" alt="description" width="100%%">
            </div>
            
            <div id="Sintesi%d" class="vtabcontent%d">
%s
            </div>
        </div>
'''

html_games = '''
        <div id="Partite" class="tabcontent">
          <h4>
            <ul>
              <table>
'''


html_game1 = '''
                <tr onclick="openContent(event, 'p%d')"><td align="right" width="3%%"><font color="#BB1010">%d.a&nbsp</font></td>
                                                        <td width="5%%"><font color="#BB1010">%s&nbsp</font></td>
                                                        <td width="2%%"><font color="#BB1010">%s&nbsp</font></td>
                                                        <td width="5%%"><font color="#BB1010">%s&nbsp</font></td>
                                                        <td width="5%%"><font color="#BB1010">%s&nbsp&nbsp</font></td>
                                                        <td width="70%%"><font color="%s"><b>%s</b></font></td>
                                                        <td align="left" width="10%%"><font color="%s"><b>%s</b></font></td></tr>'''

html_game2 = '''
                <tr onclick="openContent(event, 'p%d')"><td colspan="2" width="8%%"></td>
                                                        <td colspan="3" width="12%%" valign=top><font size=2>%s</font></td>
                                                        <td width="70%%"><font size=2>%s</font></td>
                                                        <td width="10%%"></td></tr>
                <tr onclick="openContent(event, 'p%d')"><td colspan="2" width="8%%"></td>
                                                        <td colspan="3" width="12%%" valign=top><font size=2>%s</font></td>
                                                        <td width="70%%"><font size=2>%s</font></td>
                                                        <td width="10%%"></td></tr>
                <tr><td height="10"></td></tr>
'''

html_tail = '''
              </table>
            </ul>
          </h4>
        </div>

<br>
        <div id="footer">
            <p>&copy; Copyright (C) 2011-2026 pyBasket</p>
        </div>
    </div>

    <script>
        function openContent(evt, name) {
            var i, tabcontent, tablinks;
            tabcontent = document.getElementsByClassName("tabcontent");
            for (i = 0; i < tabcontent.length; i++) {
              tabcontent[i].style.display = "none";
            }
            tablinks = document.getElementsByClassName("tablinks");
            for (i = 0; i < tablinks.length; i++) {
               tablinks[i].className = tablinks[i].className.replace(" active", "");
            }
            document.getElementById(name).style.display = "block";
            evt.currentTarget.className += " active";
        }

        // Get the element with id="defaultOpen" and click on it
        document.getElementById("defaultOpen").click();

        includeHTML();

        %s
    </script>
</body>
</html>
'''

colWin  = '#008800'
colLost = '#AA0000'
colNone = '#777777'

htmlfile = open('web/indexNEW.html', 'w')

sss = ''
for i in range(1,len(allfiles)+1):
    sss += html_vtabcontent%i

    
fff = ''
for i in range(1,len(allfiles)+1):
    fff += html_function%(i,i,i,i)

    
htmlfile.write(html_head%(sb.game.team_data['season'], sb.game.team_data['name'], sss, sb.game.team_data['name'], sb.game.team_data['championship'], sb.game.team_data['season']))

partite = ''

progressive = 1
for phase in phases:

    # File of the phase
    files = [x for x in allfiles if phase in x and os.path.basename(x).split('-')[1] == phase]

    # Sort on increasing phase + round
    rounds = [toint(os.path.basename(x).split('.')[0]) for x in files]
    files  = [file for r, file in sorted(zip(rounds, files))]

    for file in files:
        sb.game.loadGame(file)
        g = sb.game.game_data
        
        # Save Box Score in SVG format
        svg = BoxScore.svg(sb.game.events_df, game=sb.game, width=65.0)
        with open('web/%d.svg'%progressive, 'w') as outfile:
            outfile.write(svg)

        # Save Points Chart in PNG format
        fig = BoxScore.pointsChart(sb.game.events_df, game=sb.game)
        bbb = fig.to_image('png', width=2000, height=700)
        with open('web/%d.png'%progressive, 'wb') as pngfile:
            pngfile.write(bbb)

        # Save Play-By-Play directly in the HTML
        pbp = BoxScore.play_by_play(sb.game.events_df, game=sb.game)
        pbphtml = '<div style="max-width: 100%%; overflow: hidden; background-color: #ffffff;">%s</div>'%(pbp)
        with open('web/%d.html'%progressive, 'w') as pbpfile:
            pbpfile.write(pbphtml)
        
        # Save game summary
        summary = '''
                <p style="font-size: 1.3em; margin-left: 20px; color: black;">
                %s
                </p>
''' % (BoxScore.summary(sb.game.events_df, game=sb.game).replace('\n','</br>'))
        
        htmlfile.write(html_singlegame%(progressive,progressive,progressive,progressive,progressive,progressive,progressive,progressive,
                                        progressive,progressive,progressive,progressive,progressive,progressive,progressive,progressive,progressive,progressive,
                                        progressive,progressive,progressive,progressive,
                                        progressive,
                                        progressive,progressive,progressive,progressive,progressive,progressive,progressive,
                                        summary))
        
        # Accumulate minutes and plusminus
        for player in sb.game.players_info.keys():
            pi = sb.game.players_info[player]
            if player in players_info:
                players_info[player]['time_on_field'] += pi['time_on_field']
                players_info[player]['plusminus']     += pi['plusminus']
            else:
                players_info[player] = pi
        
        # Append all events
        df = sb.game.events_df
        if df.shape[0] > 0:
            name = file.replace('./data/','').replace('.game','')
            elems = name.split('-')
            ro = int(elems[0].replace('.a',''))
            ph = elems[1]
            op = elems[2]
            df['game_number'] = progressive
            df['round'] = ro
            df['phase'] = ph
            df['opponents'] = op
            df['home'] = sb.game.game_data['home']
            allevents.append(df)
            
        # Generate text for html
        pt = []
        po = []
        for player_name in sb.game.players_by_number:
            points = Stats.points(df, player_name)
            if sb.game.players_info[player_name]['time_on_field'] <= 0.0:
                pt.append('%s ne'%player_name)
            else:
                if points > 0: pt.append('%s %d'%(player_name, points))
                else:          pt.append(player_name)

        for player_name in sb.game.opponents_by_number:
            points = Stats.points(df, player_name, team=Config.OPPO)
            if points > 0: po.append('%s %d'%(player_name,points))
            else:          po.append(player_name)
            
        fint = Stats.points(df)
        fino = Stats.points(df, team=Config.OPPO)
        if g['home']:
            t1 = sb.game.team_data['name']
            t2 = g['opponents']
            r1 = t1 + ' - ' + t2
            r2 = str(fint) + '-' + str(fino)
            p1 = ', '.join(pt)
            p2 = ', '.join(po)
        else:
            t1 = g['opponents']
            t2 = sb.game.team_data['name']
            r1 = t1 + ' - ' + t2
            r2 = str(fino) + '-' + str(fint)
            p1 = ', '.join(po)
            p2 = ', '.join(pt)
            
        showpoints = True
        if fint > fino:
            col = colWin
        elif fint < fino:
            col = colLost
        else:
            col = colNone
            showpoints = False
            r2 = ''
            
        d = datetime.datetime.strptime(g['date'], "%d/%m/%Y")
        day = ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì', 'Sabato', 'Domenica'][d.weekday()]
        partite += html_game1%(progressive, g['round'], g['phase'], day, g['date'], g['time'], col, r1, col, r2)
        
        if showpoints:
            partite += html_game2%(progressive, t1, p1, progressive, t2, p2)
        
        progressive += 1

htmlfile.write(html_games)
htmlfile.write(partite)
htmlfile.write(html_tail%fff)
htmlfile.close()

df = pd.concat(allevents)
df.reset_index(drop=True, inplace=True)
df

Output()

,team,player,event,event_name,event_description,quarter,seconds,x,y,time,game_number,round,phase,opponents,home
0,Team,Di Francesco,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO,False
1,Team,Colotti,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO,False
2,Team,Dusels,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO,False
3,Team,Aloi,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO,False
4,Team,Diana,18,Entr,Entrata in campo,1,600.0,0.0,0.0,2025-02-09 17:11:58,1,1,Andata,PORTO S.ELPIDIO,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5372,Team,Diana,19,Usci,Uscita dal campo,4,0.0,0.0,0.0,2025-02-09 18:39:52,17,4,Ritorno,GIOVANE ROBUR OSIMO,True
5373,Team,Aloi,19,Usci,Uscita dal campo,4,0.0,0.0,0.0,2025-02-09 18:39:52,17,4,Ritorno,GIOVANE ROBUR OSIMO,True
5374,Team,Colotti,19,Usci,Uscita dal campo,4,0.0,0.0,0.0,2025-02-09 18:39:52,17,4,Ritorno,GIOVANE ROBUR OSIMO,True
5375,Team,Pentucci,19,Usci,Uscita dal campo,4,0.0,0.0,0.0,2025-02-09 18:39:52,17,4,Ritorno,GIOVANE ROBUR OSIMO,True


In [165]:
importlib.reload(BoxScore)

svg = BoxScore.totalsvg(df, game=sb.game, average=False, players_info=players_info, width=55.0)
with open('web/totali.svg', 'w') as file:
    file.write(svg)

svg = BoxScore.totalsvg(df, game=sb.game, average=True, players_info=players_info, width=55.0)
with open('web/medie.svg', 'w') as file:
    file.write(svg)
    
HTML(svg)

HTML(value='<svg version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" x…